# Artificial Neural Networks

ip-biocode $\cdot$ March 26, 2022

In this exercise, I build a neural network from scratch and code how it performs predictions using forward propagation. This is only to help with understanding the underlying mechanisms. All deep learning libraries have the entire training and prediction processes implemented, and so in practice you wouldn't really need to build a neural network from scratch.

## Forward Propagation: Simple Example

<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width=600px>

Here is a simple neural network that takes two inputs, has one hidden layer with two nodes, and an output layer with one node.

### Initialize weights and biases
Begin by randomly initializing weights and biases in the network. We have 6 weights and 3 biases.

In [55]:
import numpy as np

weights = np.around(np.random.uniform(size=6), decimals=2)
biases = np.around(np.random.uniform(size=3), decimals=2)

print(weights)
print(biases)

[0.92 0.9  0.03 0.96 0.14 0.28]
[0.61 0.94 0.85]


### Compute weighted sums
Now that we have the weights and the biases defined for the network, let's compute the output for a given input, $x\_1$ and $x\_2$.


In [3]:
x_1 = 0.5 # input 1
x_2 = 0.85 # input 2

print('x1 is {} and x2 is {}'.format(x_1, x_2))

x1 is 0.5 and x2 is 0.85


Compute the wighted sum of the inputs at the first node of the hidden layer, $z_{1, 1}$. The weights are $w_1$ and $w_2$.


In [8]:
z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_11))

The weighted sum of the inputs at the first node in the hidden layer is 0.9219999999999999


Next compute the weighted sum of the inputs at the second node of the hidden layer, $z_{1, 2}$. The weights are $w_3$ and $w_4$.


In [10]:
z_12 = x_1 * weights[2] + x_2 * weights[3] + biases[1]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_12))

The weighted sum of the inputs at the first node in the hidden layer is 0.9055


### Compute node activation
Next, assuming a sigmoid activation function, compute the activation of the first node, $a_{1, 1}$, in the hidden layer.


In [11]:
a_11 = 1.0 / (1.0 + np.exp(-z_11))

print('The activation of the first node in the hidden layer is {}'.format(np.around(a_11, decimals=4)))

The activation of the first node in the hidden layer is 0.7154


Compute the activation of the second node, $a_{1, 2}$, in the hidden layer.


In [12]:
a_12 = 1.0 / (1.0 + np.exp(-z_12))

print('The activation of the first node in the hidden layer is {}'.format(np.around(a_12, decimals=4)))

The activation of the first node in the hidden layer is 0.7121


Now these activations will serve as the inputs to the output layer. Compute the weighted sum of these inputs to the node in the output layer.

In [13]:
z_2 = z_11*weights[4] + z_12*weights[5] + biases[2]

print('The weighted sum of the inputs at the node in the output layer is {}'.format(np.around(z_2, decimals=4)))

The weighted sum of the inputs at the node in the output layer is 1.0397


Finally, compute the output of the network as the activation of the node in the output layer.


In [14]:
a_2 = 1.0 / (1.0 + np.exp(-z_2))

print('The output of the network for x1=0.5 and x2=0.85 is {}'.format(np.around(a_2, decimals=4)))

The output of the network for x1=0.5 and x2=0.85 is 0.7388


## A General Network
Obviously, neural networks for real problems are composed of many hidden layers and many more nodes in each layer. So, we can't continue making predictions using this very inefficient approach of computing the weighted sum at each node and the activation of each node manually.

In order to make predictions automatically, we should generalize our network. A general network would take $n$ inputs, would have many hidden layers, each hidden layer having $m$ nodes, and would have an output layer. Although the network is showing one hidden layer, we will code the network to have many hidden layers. Similarly, although the network shows an output layer with one node, we will code the network to have more than one node in the output layer.

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width=600px>




### Initialize network

Define structure of the network.

In [16]:
n = 2 # inputs
num_hidden_layers = 2 # hidden layers
m = [2, 2] # nodes in each hidden layer
num_nodes_output = 1 # nodes in output layer

Initialize weights and biases with random numbers.

In [19]:
num_nodes_previous = n # nodes in the previous layer

network = {} # initialize network as an empty dictionary

# loop through each layer
# Adding 1 to number of hidden layers for output layer
for layer in range(num_hidden_layers + 1): 
    
    # assign name of layer
    if layer == num_hidden_layers:
        layer_name = 'output'
        num_nodes = num_nodes_output
    else:
        layer_name = 'layer_{}'.format(layer + 1)
        num_nodes = m[layer]
    
    # initialize weights and biases in current layer
    network[layer_name] = {}
    for node in range(num_nodes):
        node_name = 'node_{}'.format(node+1)
        network[layer_name][node_name] = {
            'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
            'bias': np.around(np.random.uniform(size=1), decimals=2),
        }
    
    num_nodes_previous = num_nodes
    
print(network)

{'layer_1': {'node_1': {'weights': array([0.34, 0.79]), 'bias': array([0.85])}, 'node_2': {'weights': array([0.37, 0.72]), 'bias': array([0.18])}}, 'layer_2': {'node_1': {'weights': array([0.49, 0.86]), 'bias': array([0.83])}, 'node_2': {'weights': array([0.42, 0.96]), 'bias': array([0.64])}}, 'output': {'node_1': {'weights': array([0.09, 0.04]), 'bias': array([0.29])}}}


Now we are able to initialize the weights and the biases pertaining to any network of any number of hidden layers and number of nodes in each layer. I will put this code in a function so that we are able to repetitively execute all this code whenever we want to construct a neural network.




In [36]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
    
    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}
    
    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):
        
        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer] 
        
        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }
    
        num_nodes_previous = num_nodes

    return network # return the network

#### Use the *initialize_network* function to create a network that:

1.  takes 5 inputs
2.  has three hidden layers
3.  has 3 nodes in the first layer, 2 nodes in the second layer, and 3 nodes in the third layer
4.  has 1 node in the output layer

Call the network **small_network**.


In [37]:
small_network = initialize_network(5, 3, [3, 2, 3], 1)

### Weighted sums

The weighted sum at each node is computed as a dot product of the inputs and the weights plus the bias. So let's create a function called *compute_weighted_sum* that does just that.


In [56]:
def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs*weights) + bias

First generate random input values for the five inputs.

In [57]:
from random import seed

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


Define weights and bias, and then compute weighted sum.

In [58]:
 small_network['layer_1']['node_1']['bias']

array([0.28])

In [60]:
node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']

weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias)

print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(weighted_sum[0], decimals=4)))

The weighted sum at the first node in the hidden layer is 1.602


### Compute node activation

Recall that the output of each node is simply a non-linear tranformation of the weighted sum. We use activation functions for this mapping. Let's use the sigmoid function as the activation function here. So let's define a function that takes a weighted sum as input and returns the non-linear transformation of the input using the sigmoid function.


In [61]:
def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

In [65]:
node_output = node_activation(weighted_sum)

print('The output of the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))

The output of the first node in the hidden layer is 0.8323


### Forward propagation

The final piece of building a neural network that can perform predictions is to put everything together. So let's create a function that applies the *compute_weighted_sum* and *node_activation* functions to each node in the network and propagates the data all the way to the output layer and outputs a prediction for each node in the output layer.

We will follow the following procedure:

1.  Start with the input layer as the input to the first hidden layer.
2.  Compute the weighted sum at the nodes of the current layer.
3.  Compute the output of the nodes of the current layer.
4.  Set the output of the current layer to be the input to the next layer.
5.  Move to the next layer in the network.
6.  Repeat steps 2 - 4 until we compute the output of the output layer.


In [66]:
def forward_propagate(network, inputs):
    
    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer
    
    for layer in network:
        
        layer_data = network[layer]
        
        layer_outputs = [] 
        for layer_node in layer_data:
        
            node_data = layer_data[layer_node]
        
            # compute the weighted sum and the output of each node at the same time 
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))
            
        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))
    
        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

#### Use the *forward_propagate* function to compute the prediction of our small network


In [69]:
predictions = forward_propagate(small_network, inputs)
print('The predicted value by the network for the given input is {}'.format(np.around(predictions[0], decimals=4)))


The outputs of the nodes in hidden layer number 1 is [0.8323, 0.8268, 0.7735]
The outputs of the nodes in hidden layer number 2 is [0.7932, 0.8991]
The outputs of the nodes in hidden layer number 3 is [0.8232, 0.8924, 0.8141]
The predicted value by the network for the given input is 0.9112


## Summary

We built the code to define a neural network from scratch. Now we can specify the number of inputs that a neural network can take, the number of hidden layers as well as the number of nodes in each hidden layer, and the number of nodes in the output layer.

We first use the *initialize_network* to create our neural network and define its weights and biases.

In [71]:
my_network = initialize_network(5, 3, [2, 3, 2], 3)

Then for a given input, we compute the network predictions.

In [72]:
inputs = np.around(np.random.uniform(size=5), decimals=2)

In [73]:
predictions = forward_propagate(my_network, inputs)
print('The predicted values by the network for the given input are {}'.format(predictions))

The outputs of the nodes in hidden layer number 1 is [0.8857, 0.8889]
The outputs of the nodes in hidden layer number 2 is [0.7822, 0.6965, 0.7411]
The outputs of the nodes in hidden layer number 3 is [0.868, 0.881]
The predicted values by the network for the given input are [0.8952, 0.8222, 0.8035]
